# Solving Sudoku Puzzles
## Assignment Preamble
Please ensure you carefully read all of the details and instructions on the assignment page, this section, and the rest of the notebook. If anything is unclear at any time please post on the forum or ask a tutor well in advance of the assignment deadline.

In addition to all of the instructions in the body of the assignment below, you must also follow the following technical instructions for all assignments in this unit. *Failure to do so may result in a grade of zero.*
* [At the bottom of the page](#Submission-Test) is some code which checks you meet the submission requirements. You **must** ensure that this runs correctly before submission.
* Do not modify or delete any of the cells that are marked as test cells, even if they appear to be empty.
* Do not duplicate any cells in the notebook – this can break the marking script. Instead, insert a new cell (e.g. from the menu) and copy across any contents as necessary.

Remember to save and backup your work regularly, and double-check you are submitting the correct version.

This notebook is the primary reference for your submission. You may write code in separate `.py` files but it must be clearly imported into the notebook so that it runs without needing to reference those files, and you must explain clearly what functionality is contained in those files (through comments, markdown cells, etc).

As always, **the work you submit for this assignment must be entirely your own.** Do not copy or work with other students. Do not copy answers that you find online. These assignments are designed to help improve your understanding first and foremost – the process of doing the assignment is part of *learning*. They are also used to assess your ability, and so you must uphold academic integrity. Submitting plagiarised work risks your entire place on your degree.

**The pass mark for this assignment is 40%.** We expect that students, on average, will be able to produce a submission which gets a mark between 50-70% within the normal workload allocation for the unit, but this will vary depending on individual backgrounds. Please ask for help if you are struggling.

## Getting Started
For this assignment, you will be writing an agent that can solve sudoku puzzles. You should be familiar with sudoku puzzles from the unit material. You are given a 9x9 grid with some fixed values. To solve the puzzle, the objective is to fill the empty cells of the grid such that the numbers 1 to 9 appear exactly once in each row, column, and 3x3 block of the grid. 

Below is a sample puzzle along with its solution. 

<img src="./images/sudoku.png" style="width: 50%;"/>

For this assignment you will need to submit:
1. The implementation for an agent which can solve sudoku puzzles – this notebook
 * You can use any algorithm you like, from the unit material or otherwise
 * Your code will be subject to automated testing, from which grades will be assigned based on whether it can solve sudokus of varying difficulty
 * To get a high grade on this assignment, the speed of your code will also be a factor – the quicker the better
 * There are some sample tests included below, make sure your code is compatible with the format of these tests
2. A text file that explains your approach and the decisions you made in your own words – a readme file
 * Submissions that do not include the written section will receive zero marks – **this part is mandatory**
 * You may write your file in plain text (.txt) or [Markdown](https://www.markdownguide.org/basic-syntax/) (.md)
 * To get top marks on this assignment, as well as getting a high grade from your implementation, you must also demonstrate excellent academic presentation in your written section

### Choice of Algorithm
The choice of algorithm to solve sudoku puzzles is up to you. We expect you will use search techniques from the unit, but you could make something up yourself, or do some independent research to find something else. You will need to evaluate and balance the trade-off between how well suited you think the algorithm is and how difficult it is to write, but there is some advice below.

I suggest you implement *constraint satisfaction* as it is described in the unit material. You can use the code you have previously been given as a guide. A good implementation of a backtracking depth-first search with constraint propagation should be sufficient to get a good grade in the automated tests (roughly 60-70%).

You could also write a successful agent that uses the other search techniques you have seen in the unit so far: basic search, heuristic search, or local search. You may find these easier to implement, though they may perform less well. 

To get a high grade on this assignment will require a particularly efficient implementation of constraint satisfaction, or something which goes beyond the material we have presented. *This is left unguided and is not factored into the unit workload estimates.*

If you choose to implement more than one algorithm, please feel free to include your code and write about it in part two (readme file), but only the code in this notebook will be used in the automated testing.

## Sample Sudoku Puzzles
To get started, the cell below will load in some sample sudoku puzzles for you so you can see the format. There are sudokus provided of multiple difficulties (easier sudokus typically start with more digits provided). The cell below only loads the easiest, but there is another test cell lower in the notebook which will run your code against all of the provided puzzles.

Each sudoku is a 9x9 NumPy array of integers, where zero represents an empty square. Each difficulty comes with 15 sudokus, so when you load the file, it is stored in a 15x9x9 array.

In [1]:
import numpy as np

# Load sudokus
from typing import Tuple, List
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

very_easy_puzzle.npy has been loaded into the variable sudoku
sudoku.shape: (15, 9, 9), sudoku[0].shape: (9, 9), sudoku.dtype: int8

First sudoku:
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]] 

Solution of first sudoku:
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]


## Part One
You should write all of your code for solving sudokus below this cell.

You must include a function called `sudoku_solver(sudoku)` which takes one sudoku puzzle (a 9x9 NumPy array) as input, and returns the solved sudoku as another 9x9 NumPy array. This is the function which will be tested. 

In [2]:
def print_2d_list(collection: list):
    print("[")
    for i in range(len(collection)):
        print(collection[i])
    print("]")

In [3]:
"""
Given i (either a column or row position) it finds the range for the column/row part of the box
Useful to find which box to check for a number given a position

:param i: position value (somewhere in the range 1 to 9)
"""
def find_box_range(i: int) -> List[int]:
    if 0 <= i < 3:
        return [0, 1, 2]
    if 3 <= i < 6:
        return [3, 4, 5]
    else:
        return [6, 7, 8]

def in_box(board: np.array, pos: Tuple[int, int], n: int) -> bool:
    r, c = pos
    range_r = find_box_range(r)
    range_c = find_box_range(c)

    for row in range_r:
        for column in range_c:
            if row == r and column == c:
                continue

            if board[row][column] == n:
                return True
    return False

def in_horizontal(board: np.array, pos: Tuple[int, int], n: int) -> bool:
    r,c = pos
    # Search entire row
    for column in range(9):
        if column == c:
            continue
        # Number is in this row
        if board[r][column] == n:
            return True
    return False

def in_vertical(board: np.array, pos: Tuple[int, int], n: int) -> bool:
    r, c = pos
    # Search entire column
    for row in range(9):
        if row == r:
            continue
        # Number is in this column
        if board[row][c] == n:
            return True
    return False

def is_valid_pos(board: np.array, pos: Tuple[int, int], n: int) -> bool:
    return not in_box(board, pos, n) and not in_horizontal(board, pos, n) and not in_vertical(board, pos, n)

In [4]:
def next_pos(board: np.array) -> Tuple[int, int]:
    for row in range(len(board)):
        for column in range(len(board[0])):
            # if free cell, that is the next cell to explore
            if board[row][column] == 0:
                return row, column
    return

def still_have_unassigned_cells(board: np.array) -> bool:
    for row in range(len(board)):
        for column in range(len(board[0])):
            # if free cell, that is the next cell to explore
            if board[row][column] == 0:
                return True
    return False

def next_possible_pos(board: np.array, possible_actions_board: list) -> Tuple[int, int]:
    for row in range(len(possible_actions_board)):
        for column in range(len(possible_actions_board[0])):
            if board[row][column] == 0:
                if len(possible_actions_board[row][column]) > 0:
                    return row, column

    return None

In [5]:
def deal_with_1_possible_action(board: np.array, possible_actions_board: list, pos: Tuple[int, int]) -> Tuple[np.array, list]:
    new_board = np.copy(board)
    new_possible_actions_board = list.copy(possible_actions_board)

    r, c = pos
    only_remaining_option = new_possible_actions_board[r][c][0]

    # If the only remaining option is valid, then pick it, otherwise return an unchanged board and empty possible action board
    if is_valid_pos(new_board, pos, only_remaining_option):
        new_board[r][c] = only_remaining_option
        new_possible_actions_board[r][c] = []
        # Propagate the effect of the assignment
        return propagate(new_board, new_possible_actions_board, (r,c), only_remaining_option)
    # Position is not valid
    else:
        new_possible_actions_board[r][c] = []
        return new_board, new_possible_actions_board

def propagate_horizontally(board: np.array, possible_actions_board: list, pos: Tuple[int, int], n: int) -> Tuple[np.array, list]:
    new_board = np.copy(board)
    new_possible_actions_board = list.copy(possible_actions_board)
    # Get the row and column of the position
    r, c = pos
    indexes_to_check = [*range(9)]
    indexes_to_check.pop(c)

    for i in indexes_to_check:
        try:
            # If we find n in the array of possible actions, remove it
            index_of_n = new_possible_actions_board[r][i].index(n)
            new_possible_actions_board[r][i].pop(index_of_n)
            # If we only have an option for an action, pick that action
            if len(new_possible_actions_board[r][i]) == 1:
                new_board, new_possible_actions_board = deal_with_1_possible_action(new_board, new_possible_actions_board, (r,i))

        # If couldn't find n we dont need to do anything
        except ValueError:
            pass

    return new_board, new_possible_actions_board

def propagate_vertically(board: np.array, possible_actions_board: list, pos: Tuple[int, int], n: int) -> Tuple[np.array, list]:
    new_board = np.copy(board)
    new_possible_actions_board = list.copy(possible_actions_board)

    # Get the row and column of the position
    r, c = pos
    indexes_to_check = [*range(9)]
    indexes_to_check.pop(r)
    for i in indexes_to_check:
        try:
            # If we find n in the array of possible actions, remove it
            new_possible_actions_board[i][c].pop(new_possible_actions_board[i][c].index(n))
            # If we only have an option for an action, pick that action
            if len(new_possible_actions_board[i][c]) == 1:
                new_board, new_possible_actions_board = deal_with_1_possible_action(new_board, new_possible_actions_board, (i,c))

        # If couldn't find n we dont need to do anything
        except ValueError:
            pass

    return new_board, new_possible_actions_board

def propagate_box_wise(board: np.array, possible_actions_board: list, pos: Tuple[int, int], n: int) -> Tuple[np.array, list]:
    new_board = np.copy(board)
    new_possible_actions_board = list.copy(possible_actions_board)
    
    r, c = pos
    range_r = find_box_range(r)
    range_c = find_box_range(c)

    for row in range_r:
        for column in range_c:
            if row==r and column==c:
                continue
            try:
                # If we find n in the array of possible actions, remove it
                new_possible_actions_board[row][column].pop(new_possible_actions_board[row][column].index(n))
                # If we only have an option for an action, pick that action
                if len(new_possible_actions_board[row][column]) == 1:
                    new_board, possible_actions_board = deal_with_1_possible_action(new_board, new_possible_actions_board, (row,column))

                break

            # If couldn't find n we dont need to do anything
            except ValueError:
                pass
    return new_board, new_possible_actions_board

def propagate(board: np.array, possible_actions_board: list, pos: Tuple[int, int], n: int) -> Tuple[np.array, list]:
    board , possible_actions_board = propagate_horizontally(board, possible_actions_board, pos, n)
    board , possible_actions_board = propagate_vertically(board, possible_actions_board, pos, n)
    board , possible_actions_board = propagate_box_wise(board, possible_actions_board, pos, n)
    return board, possible_actions_board

In [6]:
def old_solver(sudoku):
    if not is_board_valid(sudoku):
        return np.full((9,9), -1)

    next_position = next_pos(sudoku)
    # If next_position is None then we have reached the end of the sudoku, meaning we WON
    if next_position is None :
        return sudoku

    row, column =  next_position
    # For each cell we try values 1 to 9 inclusively
    for n in range(1, 10):
        # Can we insert the value in the cell
        if is_valid_pos(sudoku, next_position, n):
            # Assign the cell we are exploring to the value
            sudoku[row][column] = n

            new_sudoku = old_solver(sudoku)
            # If reached a state where we need to backtrack
            if np.array_equal(new_sudoku, np.full((9,9), -1)):
                sudoku[row][column] = 0
            # If sudoku solved
            else:
                return new_sudoku

    return np.full((9,9), -1)

In [7]:
def is_board_valid(board: np.array) -> bool:
    for row in range(len(board[0])):
        for column in range(len(board[0])):
            # if found a cell that is above 0, we have to check if its position is valid
            cell_value = board[row][column]
            if cell_value > 0:
                if not is_valid_pos(board, (row, column), cell_value):
                    return False
    return True

def init_possible_actions_board(board: np.array) -> list:
    possible_actions_board = []
    for row in range(len(board[0])):
        column_list = []
        for column in range(len(board[0])):
            if board[row][column] == 0:
                column_list.append([1, 2, 3, 4, 5, 6, 7, 8, 9])
            else:
                column_list.append([])
        possible_actions_board.append(column_list)
    return possible_actions_board

def initial_propagation(board: np.array, possible_actions_board: list) -> Tuple[np.array, list]:
    new_board = np.copy(board)
    new_possible_actions_board = list.copy(possible_actions_board)
    for row in range(len(new_board)):
        for column in range(len(new_board[0])):
            # If defined
            temp_value = new_board[row][column]
            if temp_value != 0:
                new_board, new_possible_actions_board = propagate(new_board, new_possible_actions_board, (row, column), temp_value)
    return new_board, new_possible_actions_board

def solve_sudoku(board: np.array, possible_actions_board: list) -> np.array:

    new_board = np.copy(board)
    new_possible_actions_board = list.copy(possible_actions_board)
    
    next_position = next_pos(board)
    # If next_position is None then we have no remaining moves
    if next_position is None :
            return board

    row, column = next_position
    possible_actions_for_pos = possible_actions_board[row][column]
    if len(possible_actions_for_pos) == 0:
        print("------------")
        print("Board")
        print_2d_list(new_board)
        print("Possible_Actions_Board")
        print_2d_list(new_possible_actions_board)
        print("------------")
        print()
        print("FAILED BECAUSE NO MOVES LEFT: ["+ str(row) + "][" + str(column) +"]")
        return np.full((9,9), -1)

    for n in possible_actions_for_pos:

        # Assign the cell we are exploring to the value
        new_board[row][column] = n
        new_possible_actions_board[row][column].pop(0)

        # Propagate the effect of the assignment
        new_board, new_possible_actions_board = propagate(new_board, new_possible_actions_board, (row, column), n)

        print("------------")
        print("Board")
        print_2d_list(new_board)
        print("Possible_Actions_Board")
        print_2d_list(new_possible_actions_board)
        print("------------")
        print()

        new_board = solve_sudoku(new_board, new_possible_actions_board)
        # If reached a state where we need to backtrack
        if np.array_equal(new_board, np.full((9,9), -1)):
            new_board[row][column] = 0
        # If sudoku solved
        else:
            return new_board

    return np.full((9,9), -1)

In [8]:
def sudoku_solver(sudoku):
    """
    Solves a Sudoku puzzle and returns its unique solution.

    Input
        sudoku : 9x9 numpy array
            Empty cells are designated by 0.

    Output
        9x9 numpy array of integers
            It contains the solution, if there is one. If there is no solution, all array entries should be -1.
    """

    if not is_board_valid(sudoku):
        return np.full((9,9), -1)

    possible_actions_board = init_possible_actions_board(sudoku)

    sudoku, possible_actions_board = initial_propagation(sudoku, possible_actions_board)

    solved_sudoku = solve_sudoku(sudoku, possible_actions_board)

    return solved_sudoku

In [9]:
sudoku = np.load("data/easy_puzzle.npy")
print(sudoku[4])
print(initial_propagation(sudoku[4], init_possible_actions_board(sudoku[4])))

[[0 2 4 7 6 3 5 9 1]
 [3 9 7 4 5 0 8 2 6]
 [8 5 6 9 2 0 0 4 7]
 [0 8 9 5 3 6 7 1 4]
 [7 3 1 2 9 4 6 5 8]
 [6 4 5 8 1 7 9 3 2]
 [4 7 2 3 8 9 1 6 5]
 [9 1 8 6 4 5 2 7 3]
 [5 6 3 1 7 2 4 8 9]]
(array([[0, 2, 4, 7, 6, 3, 5, 9, 1],
       [3, 9, 7, 4, 5, 1, 8, 2, 6],
       [8, 5, 6, 9, 2, 0, 3, 4, 7],
       [2, 8, 9, 5, 3, 6, 7, 1, 4],
       [7, 3, 1, 2, 9, 4, 6, 5, 8],
       [6, 4, 5, 8, 1, 7, 9, 3, 2],
       [4, 7, 2, 3, 8, 9, 1, 6, 5],
       [9, 1, 8, 6, 4, 5, 2, 7, 3],
       [5, 6, 3, 1, 7, 2, 4, 8, 9]], dtype=int8), [[[], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], []]])


All of your code must go above this cell. You may add additional cells into the notebook if you wish, but do not duplicate or copy/paste cells as this can interfere with the grading script.

### Testing Details
There are four difficulties of sudoku provided: very easy, easy, medium, and hard. There are 15 sample sudokus in each category, with solutions as well. Difficulty was determined using reference solvers, but your code may vary; it is conceivable that your code will find some sudokus much easier or harder within a given category, or even between categories.

*All categories that are easy and above will contain* ***invalid initial states***, that is, sudoku puzzles with no solution. In this case, your function should return a 9x9 NumPy array whose values are all equal to -1.

When we test your code, we will firstly test it on the *same* very easy puzzles that you have been given. Then we will test it on additional *hidden* sudokus from each difficulty in turn, easy and up. Grades are awarded based on whether your code can solve the puzzles. For high grades on the hard puzzles, execution time will also be a factor. 

All puzzles must take under 30 seconds each on the test machine to count as successful, but you should be aiming for an average of under a second per puzzle. Hardware varies, but all tests will take place on the same modern desktop machine. Our ‘standard constraint satisfaction’ implementation takes about 0.001 seconds per puzzle for the very easy category, but struggles to solve some of the hard puzzles within the time limit.

***The hard sudokus are labelled as hard for a reason.*** We expect most submissions will not be able to solve them in a reasonable length of time. Use the stop button (■) on the toolbar if you need to terminate your code because it is taking too long.

The best way to improve the performance of your code is through a detailed understanding and smart choice of AI algorithms. This assignment is ***not*** meant to test your ability to write multi-threaded code or any other kind of high-performance code optimisations. 

#### Test Cell
The following code will run your solution over the provided sudoku puzzles. To enable it, set the constant `SKIP_TESTS` to `False`. If you fail any tests of one difficulty, the code will stop, but you can modify this behaviour if you like.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [10]:
SKIP_TESTS = False

if not SKIP_TESTS:
    import time
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took", end_time-start_time, "seconds to solve.\n")

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        if count < len(sudokus):
            break

Testing very_easy sudokus
This is very_easy sudoku number 0
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]]
This is your solution for very_easy sudoku number 0
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.014920222999999955 seconds to solve.

This is very_easy sudoku number 1
[[0 9 3 1 5 2 6 0 8]
 [8 6 2 7 0 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 0 3 6]
 [5 0 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 [6 1 4 2 7 8 9 5 3]
 [7 3 9 6 1 5 8 4 2]
 [2 8 5 3 9 4 7 6 1]]
This is your solution for very_easy sudoku number 1
[[4 9 3 1 5 2 6 7 8]
 [8 6 2 7 4 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 5 3 6]
 [5 2 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]


This is your solution for easy sudoku number 10
[[8 2 6 9 3 4 1 5 7]
 [9 5 3 7 1 8 6 4 2]
 [7 4 1 2 6 5 3 9 8]
 [6 9 2 4 8 7 5 1 3]
 [1 7 5 3 2 6 9 8 4]
 [4 3 8 5 9 1 2 7 6]
 [2 8 9 1 4 3 7 6 5]
 [5 1 4 6 7 2 8 3 9]
 [3 6 7 8 5 9 4 2 1]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.007083978999999907 seconds to solve.

This is easy sudoku number 11
[[3 5 4 9 2 6 8 1 7]
 [7 2 6 1 3 8 5 9 4]
 [8 9 1 4 0 7 2 0 3]
 [1 8 5 2 9 4 3 7 6]
 [4 6 3 7 1 5 9 2 8]
 [9 7 2 6 8 3 1 4 5]
 [5 4 9 3 7 1 6 0 2]
 [6 1 8 0 4 2 7 0 9]
 [2 3 7 8 6 9 4 5 1]]
This is your solution for easy sudoku number 11
[[3 5 4 9 2 6 8 1 7]
 [7 2 6 1 3 8 5 9 4]
 [8 9 1 4 5 7 2 6 3]
 [1 8 5 2 9 4 3 7 6]
 [4 6 3 7 1 5 9 2 8]
 [9 7 2 6 8 3 1 4 5]
 [5 4 9 3 7 1 6 8 2]
 [6 1 8 5 4 2 7 3 9]
 [2 3 7 8 6 9 4 5 1]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.006438765999999929 seconds to solve.

This is easy sudoku number 12
[[4 1 5 9 6 8 2 3 7]
 [3 2 9 7 5 4 8 6 1]
 [8 7 6 1 2 3 9 5 4]

This is your solution for medium sudoku number 8
[[2 9 3 4 1 5 8 7 6]
 [8 5 6 2 7 9 3 1 4]
 [7 4 1 3 8 6 2 9 5]
 [5 2 8 6 3 1 9 4 7]
 [6 7 9 8 4 2 5 3 1]
 [3 1 4 9 5 7 6 8 2]
 [9 3 7 5 2 4 1 6 8]
 [1 8 2 7 6 3 4 5 9]
 [4 6 5 1 9 8 7 2 3]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.008936302999999923 seconds to solve.

This is medium sudoku number 9
[[3 6 2 4 0 5 0 0 7]
 [1 7 9 2 8 3 0 4 6]
 [0 5 0 6 0 7 9 3 0]
 [6 2 4 0 5 0 0 8 9]
 [5 8 0 9 7 0 4 0 3]
 [0 9 3 8 6 4 1 2 5]
 [9 3 5 1 2 0 6 7 0]
 [8 4 0 5 3 6 2 0 1]
 [2 1 6 7 4 9 3 0 8]]
This is your solution for medium sudoku number 9
[[3 6 2 4 9 5 8 1 7]
 [1 7 9 2 8 3 5 4 6]
 [4 5 8 6 1 7 9 3 2]
 [6 2 4 3 5 1 7 8 9]
 [5 8 1 9 7 2 4 6 3]
 [7 9 3 8 6 4 1 2 5]
 [9 3 5 1 2 8 6 7 4]
 [8 4 7 5 3 6 2 9 1]
 [2 1 6 7 4 9 3 5 8]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.009511292000000005 seconds to solve.

This is medium sudoku number 10
[[3 4 6 7 2 5 9 8 1]
 [0 2 7 9 4 8 5 0 6]
 [9 5 0 0 1 6 7

[0 0 0 6 0 0 0 0 0]
[0 8 0 0 0 2 0 7 0]
[5 0 7 0 0 1 0 0 0]
[0 0 0 0 0 3 6 1 0]
[7 0 0 0 0 0 2 0 9]
[0 0 0 0 5 0 0 0 0]
[3 0 0 0 0 4 0 0 5]
]
Possible_Actions_Board
[
[[], [6, 9], [6, 8, 9], [], [2, 3, 8, 9], [6, 8, 9], [3, 8, 9], [2, 3, 6, 8, 9], [2, 3, 6, 8]]
[[6, 8, 9], [], [], [1, 4, 5, 8, 9], [1, 4, 8, 9], [5, 6, 7, 8, 9], [1, 4, 5, 7, 8, 9], [4, 5, 6, 8, 9], [1, 4, 6, 7, 8]]
[[8, 9], [2, 7, 9], [1, 3, 4, 8, 9], [], [1, 2, 3, 4, 8, 9], [5, 7, 8, 9], [1, 3, 4, 5, 7, 8, 9], [2, 3, 4, 5, 8, 9], [1, 2, 3, 4, 7, 8]]
[[4, 6, 9], [], [1, 3, 4, 6, 9], [4, 5, 9], [1, 3, 4, 6, 9], [], [3, 4, 5, 9], [], [1, 3, 4]]
[[], [2, 6, 9], [], [4, 8, 9], [2, 3, 4, 6, 8, 9], [], [3, 4, 8, 9], [2, 3, 4, 8, 9], [2, 3, 4, 6, 8]]
[[2, 4, 9], [2, 7, 9], [4, 8, 9], [4, 5, 8, 9], [2, 4, 7, 8, 9], [], [], [], [2, 4, 8]]
[[], [1, 5, 6], [1, 4, 6, 8], [1, 3, 8], [1, 3, 4, 6, 8], [5, 6, 8], [], [3, 4, 6, 8], []]
[[2, 4, 6, 8, 9], [1, 2, 6, 9], [1, 4, 6, 8, 9], [1, 2, 3, 8, 9], [], [6, 7, 8, 9], [1, 3, 4, 7, 8], [

[1 0 5 8 0 0 0 0 0]
[8 0 0 9 0 0 0 0 7]
[0 0 0 7 0 0 0 3 0]
[0 0 9 0 2 0 7 0 0]
[0 2 0 0 0 0 0 0 5]
[0 0 0 0 0 0 0 0 0]
[0 4 0 2 0 8 0 6 0]
[0 0 8 4 0 5 3 7 0]
]
Possible_Actions_Board
[
[[], [], [], [], [], [], [], [], []]
[[], [3, 6], [], [], [4, 7], [2, 3, 4, 6, 7, 9], [2, 4, 6, 9], [2, 4, 9], [2, 3, 4, 6, 9]]
[[], [3, 6], [1, 3, 6], [], [4, 5], [2, 4, 6], [1, 2, 4, 6], [1, 2, 4], []]
[[4, 5, 6], [1, 5, 6, 8], [1, 2, 6], [], [1, 4, 5, 6, 8, 9], [4, 6, 9], [1, 2, 4, 6, 9], [], [1, 2, 4, 6, 8, 9]]
[[3, 4, 5, 6], [1, 3, 5, 6, 8], [], [1, 3, 5], [], [3, 4, 6], [], [1, 4, 8], [1, 4, 6, 8]]
[[3, 4, 6], [], [1, 3, 6, 7], [1, 3], [1, 4, 6, 8, 9], [3, 4, 6, 7, 9], [1, 4, 6, 9], [1, 4, 8, 9], []]
[[3, 5, 6, 9], [1, 3, 5, 6, 7], [1, 2, 3, 6, 7], [1, 3], [1, 5, 6, 7, 8, 9], [3, 4, 6, 7, 9], [1, 2, 4, 5, 9], [1, 2, 4, 8, 9], [1, 2, 4, 6, 8, 9]]
[[3, 5, 9], [], [1, 3, 7], [], [1, 7, 9], [], [1, 5, 9], [], [1, 9]]
[[6, 9], [1, 6], [], [], [1, 6, 9], [], [], [], [1, 2, 9]]
]
------------

---------

[
[[], [], [], [], [], [], [], [], []]
[[], [5, 8], [5, 9], [1, 4, 9], [1, 4, 6, 8], [], [], [5, 6, 8, 9], [4, 5, 6, 8, 9]]
[[], [2, 5], [], [4, 9], [], [2, 4, 9], [4, 5, 7, 9], [5, 7, 9], []]
[[2, 8, 9], [], [2, 4, 9], [1, 2, 4, 8, 9], [], [1, 2, 4, 9], [1, 4, 8, 9], [1, 3, 6, 8, 9], [3, 4, 6, 8, 9]]
[[], [2, 3, 5, 6, 8], [2, 4, 5, 9], [2, 4, 8, 9], [4, 8], [2, 4, 6, 9], [4, 5, 8, 9], [3, 5, 6, 8, 9], []]
[[8, 9], [5, 8], [1, 4, 5, 7, 9], [], [1, 4, 7, 8], [], [1, 4, 5, 8, 9], [], [4, 5, 8, 9]]
[[], [2, 6, 8], [1, 2, 7, 9], [1, 2, 3, 4, 7, 8], [1, 4, 6, 7, 8], [1, 2, 4, 6, 9], [1, 7, 8, 9], [1, 3, 7, 8, 9], [2, 3, 4, 6, 8, 9]]
[[2, 8, 9], [], [1, 2, 7, 9], [1, 2, 3, 7, 8], [1, 7, 8], [1, 2, 9], [], [1, 3, 5, 7, 8, 9], [2, 3, 5, 8, 9]]
[[2, 8], [2, 6, 8], [], [1, 2, 7, 8], [], [], [1, 7, 8], [], [2, 8]]
]
------------

FAILED BECAUSE NO MOVES LEFT: [0][7]
This is your solution for hard sudoku number 11
[[-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -

## Submission Test
The following cell tests if your notebook is ready for submission. **You must not skip this step!**

Restart the kernel and run the entire notebook (Kernel → Restart & Run All). Now look at the output of the cell below. 

*If there is no output, then your submission is not ready.* Either your code is still running (did you forget to skip tests?) or it caused an error.

As previously mentioned, failing to follow these instructions can result in a grade of zero.

In [11]:
import sys
import pathlib

fail = False;

if not SKIP_TESTS:
    fail = True;
    print("You must set the SKIP_TESTS constant to True in the cell above.")
    
p1 = pathlib.Path('./readme.txt')
p2 = pathlib.Path('./readme.md')
if not (p1.is_file() or p2.is_file()):
    fail = True;
    print("You must include a separate file called readme.txt or readme.md in your submission.")
    
p3 = pathlib.Path('./sudoku.ipynb')
if not p3.is_file():
    fail = True
    print("This notebook file must be named sudoku.ipynb")
    
if "sudoku_solver" not in dir():
    fail = True
    print("You must include a function called sudoku_solver which accepts a numpy array.")
else: 
    sudoku = np.load("data/very_easy_puzzle.npy")[0]
    solution = np.load("data/very_easy_solution.npy")[0]

    if not np.array_equal(sudoku_solver(sudoku), solution):
        print("Warning:")
        print("Your sudoku_solver function does not correctly solve the first sudoku.")
        print()
        print("Your assignment is unlikely to get any marks from the autograder. While we will")
        print("try to check it manually to assign some partial credit, we encourage you to ask")
        print("for help on the forum or directly to a tutor.")
        print()
        print("Please use the readme file to explain your code anyway.")
    
if fail:
    print()
    sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
else:
    print("All checks passed. When you are ready to submit, upload the notebook and readme file to the")
    print("assignment page, without changing any filenames.")
    print()
    print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")

You must set the SKIP_TESTS constant to True in the cell above.
You must include a separate file called readme.txt or readme.md in your submission.



Your submission is not ready! Please read and follow the instructions above.

In [12]:
# This is a TEST CELL. Do not delete or change.